# Introduction to Containerization with Docker

## What is containerization? What are Containers?

Containerization is the process of defining the software components required to run a *specific application*. In the context of a Docker-based containerization process this definition is written into a *Dockerfile*. A Dockerfile is then used to create a *Docker Image*. That Docker Image can then be used as the basis for one or more separate *Docker Containers*, or as the starting point for defining a new Dockerfile. Docker containers are immutable (i.e. changes to the container are **do not** update the source image, and are lost when the container is stopped *and* removed). A modified container can be used to create a new image through the *commit* command. 

![Docker flow diagram](dockerFlow.png)

This process of building a new image from a base image plus changes defined in a dockerfile, or changes made to a container based on that image and committed to a new image yields a model in which individual Docker images are made up of stacks of images that have been used to build previous base images. 

![Docker layer model](layers.png)


### How is it different from virtualization (i.e. virtual machines)?

You may have encountered references to *virtual machines* (or VMs) as a model for running virtual computer environments within/on computer hardware. A common question that arises when discussing containerization is how it differs from virtualization.  The primary difference between the two is that the machine images and corresponding virtual machines that they enable are fully self-contained virtual systems that provide all of the core capabilities (in virtual form) of a dedicated computer, including a full operating system (such as Windows or Liunx [Mac OS is infrequently used as an OS for VMs]) and set of applications running within that operating system. Containerized apps, on the other hand share a common container engine that provides core operating system capabilities that allow the application code within the container to run *as if* it were operating within a fully functional OS and hardware environment. 

![Comparison of Hypervisor VM and Container Models](VMvsContainer.png)

## Why would I use containers in my work?

* Portability of **applications**
* Comparatively lightweight
* Consistent behavior across systems
* Specific and clearly defined sectup and configuration processes through *Dockerfiles*
* Broad support
    * Cloud providers
        * [Digital Ocean](https://www.digitalocean.com)
        * [Google Cloud Platform](https://cloud.google.com/containers/)
        * [Amazon Elastic Container Service](https://aws.amazon.com/ecs/)
        * [Microsoft Azure Container Service](https://azure.microsoft.com/en-us/services/container-service/)
    * Desktop operating systems (Docker Engine & additional tools - CE)
        * [Mac OS](https://docs.docker.com/docker-for-mac/install/)
        * [Windows](https://docs.docker.com/docker-for-windows/install/)
    * Servers (CE)
        * Linux 
            * [Ubuntu](https://docs.docker.com/install/linux/docker-ce/ubuntu/)
            * [Debian](https://docs.docker.com/install/linux/docker-ce/debian/)
            * [CentOS](https://docs.docker.com/install/linux/docker-ce/centos/)
            * [Fedora](https://docs.docker.com/install/linux/docker-ce/fedora/)
* You don't need to reinvent the wheel - Many pre-configured containers have been developed and maintained by the community 
    * There are numerous container images and documentation maintained in [Docker Hub](https://hub.docker.com) and [Docker Store](https://store.docker.com/search?q=&source=community&type=image) - You can share your images through Docker Hub as sell
    * Many, many Dockerfiles maintained and accessible through [GitHub](https://github.com/search?utf8=✓&q=dockerfile&type=)
* Containers can provide a foundation for a reproducable environment & workflow
* Application containers are *imutable* and you can store data in data containers or through mounted directories on the host operating system
* Multiple containers can be "composed" into combined services

## Why would I *not* use containers?

* Your workflows and applications are primarily Windows or Mac OS application based
    * While you can run the Docker platform (i.e. Docker Engine) on a wide variety of operating systems, the applications that you can run in Docker containers are typically Linux based. 
* You typically work with applications for which there are robust installers for your particular operating system. 

## Sample Use Scenarios

### Run a Debian Linux environment

In [1]:
%%bash
# Pull the "Official" Debian docker image from Dockerhub 
docker pull debian

Using default tag: latest
latest: Pulling from library/debian
Digest: sha256:4fcd8c0b6f5e3bd44a3e63be259fd0c038476d432953d449ef34aedf16def331
Status: Image is up to date for debian:latest


    Karls-MacBook-Pro-2:~ kbene$ docker run -it debian
root@e3d7e52268a5:/# cd
root@e3d7e52268a5:~# touch demo.txt
root@e3d7e52268a5:~# ls
demo.txt
root@e3d7e52268a5:~# exit
exit
Karls-MacBook-Pro-2:~ kbene$
        
    Karls-MacBook-Pro-2:~ kbene$ docker run -it debian
root@5f4a7f5c490f:/# cd
root@5f4a7f5c490f:~# ls
root@5f4a7f5c490f:~# exit
exit
Karls-MacBook-Pro-2:~ kbene$ 

### Setup and configure a web application

Based on the official Apache HTTPD container: [https://hub.docker.com/_/httpd](https://hub.docker.com/_/httpd)

In [2]:
%%bash
cd httpd
docker build -t kbene/httpd-local .
docker run -d -p 8081:80 --name httpd-local kbene/httpd-local

Sending build context to Docker daemon   12.8kB
Step 1/2 : FROM httpd:2.4
 ---> 01154c38b473
Step 2/2 : COPY ./public-html/ /usr/local/apache2/htdocs/
 ---> Using cache
 ---> 64014fab144e
Successfully built 64014fab144e
Successfully tagged kbene/httpd-local:latest
d1e520feadabe3cd93ae33fd31e4480c066d3bfb83919d9af959ca7998bbc6b6


In [3]:
%%bash
docker ps

CONTAINER ID        IMAGE               COMMAND              CREATED                  STATUS                  PORTS                  NAMES
d1e520feadab        kbene/httpd-local   "httpd-foreground"   Less than a second ago   Up Less than a second   0.0.0.0:8081->80/tcp   httpd-local


[http://localhost:8081](http://localhost:8081)

In [4]:
%%bash
docker stop httpd-local
docker rm httpd-local
docker ps

httpd-local
httpd-local
CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


### Database server

[MySQL Official Image](https://hub.docker.com/_/mysql/)

In [5]:
%%bash
# Run the MySql server container from the official image in Dockerhub
docker pull mysql
docker run --name some-mysql -p 3306:3306 -e MYSQL_USER=demouser -e MYSQL_PASSWORD=getmein -e MYSQL_ROOT_PASSWORD=my-secret-pw -e MYSQL_DATABASE=testdb -d mysql

Using default tag: latest
latest: Pulling from library/mysql
Digest: sha256:227d5c3f54ee3a70c075b1c3013e72781564000d34fc8c7ec5ec353c5b7ef7fa
Status: Image is up to date for mysql:latest
e63d6212c683f63a9b60bffca3fef3935fc0e4989d47010e72f4641281ad9a5c


In [6]:
%%bash
docker ps

CONTAINER ID        IMAGE               COMMAND                  CREATED                  STATUS                  PORTS                    NAMES
e63d6212c683        mysql               "docker-entrypoint.s…"   Less than a second ago   Up Less than a second   0.0.0.0:3306->3306/tcp   some-mysql



    docker exec -it some-mysql bash

    root@9098f791534c:/# mysql -u demouser -p
    Enter password: 
    Welcome to the MySQL monitor.  Commands end with ; or \g.
    Your MySQL connection id is 3
    Server version: 5.7.21 MySQL Community Server (GPL)

    Copyright (c) 2000, 2018, Oracle and/or its affiliates. All rights reserved.

    Oracle is a registered trademark of Oracle Corporation and/or its
    affiliates. Other names may be trademarks of their respective
    owners.

    Type 'help;' or '\h' for help. Type '\c' to clear the current input statement.

    mysql> SHOW DATABASES;
    +--------------------+
    | Database           |
    +--------------------+
    | information_schema |
    | testdb             |
    +--------------------+
    2 rows in set (0.00 sec)

    mysql> QUIT;
    Bye
    root@9098f791534c:/# 

In [7]:
%%bash
docker stop some-mysql
docker rm some-mysql
docker ps

some-mysql
some-mysql
CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


### Desktop analysis tool - Demo

## Docker Resources

* [Documentation Home Page](https://docs.docker.com)
* [Getting Started with Docker](https://docs.docker.com/get-started/)
* [Docker Overview](https://docs.docker.com/engine/docker-overview/)
* [Command Reference](https://docs.docker.com/reference/)
    * [Command Line Interface](https://docs.docker.com/engine/reference/commandline/cli/)
    * [Dockerfile Reference](https://docs.docker.com/engine/reference/builder/)